# Multi-Scenario MobileNetV3 Transfer Learning

## Experiment Overview
This notebook implements a comprehensive transfer learning experiment using **MobileNetV3Large** for rice leaf disease classification.

### Dataset
- **Original Classes**: 10 rice leaf disease categories
- **Filtered Classes**: 4 selected classes
  - bacterial_leaf_blight
  - brown_spot
  - leaf_blast
  - healthy

### Experiment Design
**18 Training Scenarios** combining:
- **3 Data Splits**: 90:10, 80:20, 70:30 (train:validation)
- **2 Optimizers**: Adam, SGD
- **3 Learning Rates**: 0.1, 0.01, 0.001
- **3 Epoch Settings**: 15, 30, 45

### Training Configuration
| Scenario | Split Ratio | Optimizer | LR | Epochs |
|----------|-------------|-----------|-----|--------|
| 1-2 | 90:10 | Adam, SGD | 0.1 | 15 |
| 3-4 | 90:10 | Adam, SGD | 0.01 | 30 |
| 5-6 | 90:10 | Adam, SGD | 0.001 | 45 |
| 7-8 | 80:20 | Adam, SGD | 0.1 | 15 |
| 9-10 | 80:20 | Adam, SGD | 0.01 | 30 |
| 11-12 | 80:20 | Adam, SGD | 0.001 | 45 |
| 13-14 | 70:30 | Adam, SGD | 0.1 | 15 |
| 15-16 | 70:30 | Adam, SGD | 0.01 | 30 |
| 17-18 | 70:30 | Adam, SGD | 0.001 | 45 |

### Method
- **Transfer Learning**: Feature extraction with frozen MobileNetV3Large base
- **Preprocessing**: Resize to 224×224, rescale to [0,1]
- **Early Stopping**: Patience=5 epochs on validation loss
- **Architecture**: MobileNetV3Large → GlobalAvgPool → Dropout(0.5) → Dense(4)

### Outputs
1. CSV file with all scenario results
2. JSON file with training histories
3. Top 10 scenarios bar chart
4. Top 3 training curves
5. Confusion matrix for best scenario
6. Classification report for best scenario

In [1]:
import sys
import subprocess

# Reinstall protobuf with correct version
subprocess.check_call([sys.executable, "-m", "pip", "install", 
                      "--quiet", "--force-reinstall", "protobuf==3.20.3"])

import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"

print("✓ Protobuf configured successfully")

# =====================================================
# MAIN TRAINING CODE
# =====================================================
import shutil
import random
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support
from tensorflow.keras.applications import MobileNetV3Large
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import json
from datetime import datetime

tf.get_logger().setLevel('ERROR')

# Set seeds for reproducibility
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

# ------------------ 1. CONFIGURATION ------------------
class Config:
    BASE_INPUT = '/kaggle/input/rice-leaf-diseases-detection/Rice_Leaf_Diease/Rice_Leaf_Diease'
    WORK_DIR = '/kaggle/working/Rice_Leaf_Diease'
    OUTPUT_DIR = '/kaggle/working/results'
    
    # Target classes for filtering (4 out of 10 classes)
    TARGET_CLASSES = ['bacterial_leaf_blight', 'brown_spot', 'leaf_blast', 'healthy']
    
    BATCH_SIZE = 16
    IMG_SIZE_MOBILE = 224
    
    # Model architecture
    DROPOUT_RATE = 0.5
    NUM_CLASSES = 4  # Updated for 4 classes
    
    SEED = 42

config = Config()

# Create output directory for results
os.makedirs(config.OUTPUT_DIR, exist_ok=True)

print(f"Configuration loaded:")
print(f"  - Target classes: {config.TARGET_CLASSES}")
print(f"  - Number of classes: {config.NUM_CLASSES}")
print(f"  - Image size: {config.IMG_SIZE_MOBILE}x{config.IMG_SIZE_MOBILE}")
print(f"  - Batch size: {config.BATCH_SIZE}")
print(f"  - Output directory: {config.OUTPUT_DIR}")


[notice] A new release of pip is available: 25.1.1 -> 26.0
[notice] To update, run: python -m pip install --upgrade pip


✓ Protobuf configured successfully


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# ------------------ 2. DATASET FILTERING & PREPARATION ------------------

def filter_dataset_classes(source_dir, dest_dir, target_classes):
    """
    Copy only the target classes from source to destination.
    Preserves train/test directory structure.
    """
    for split in ['train', 'test']:
        source_split = os.path.join(source_dir, split)
        dest_split = os.path.join(dest_dir, split)
        
        if not os.path.exists(source_split):
            continue
            
        for class_name in target_classes:
            source_class = os.path.join(source_split, class_name)
            dest_class = os.path.join(dest_split, class_name)
            
            if os.path.exists(source_class):
                os.makedirs(dest_class, exist_ok=True)
                if not os.listdir(dest_class):  # Only copy if destination is empty
                    shutil.copytree(source_class, dest_class, dirs_exist_ok=True)
                    print(f"  Copied {class_name} from {split}/")
    
    print(f"✓ Dataset filtered to {len(target_classes)} classes")

# Copy and filter dataset
if not os.path.exists(config.WORK_DIR):
    print("Filtering and copying dataset to working directory...")
    filter_dataset_classes(config.BASE_INPUT, config.WORK_DIR, config.TARGET_CLASSES)
else:
    print("✓ Working directory already exists")

TRAIN_DIR = f'{config.WORK_DIR}/train'
TEST_DIR = f'{config.WORK_DIR}/test'

In [ ]:
# ------------------ 3. DATA SPLITTING & GENERATORS ------------------

def create_data_generators(train_dir, test_dir, train_ratio, img_size, batch_size):
    """
    Create train/validation data generators with specified split ratio.
    Merges train+test, then splits based on train_ratio.
    
    Args:
        train_dir: Original training directory
        test_dir: Original test directory (will be merged with train)
        train_ratio: Ratio for training (e.g., 0.9 for 90:10 split)
        img_size: Target image size
        batch_size: Batch size for generators
    
    Returns:
        train_generator, val_generator, class_indices
    """
    # Only rescale, no augmentation
    datagen = ImageDataGenerator(
        rescale=1./255,
        validation_split=1.0 - train_ratio  # validation_split is the % for validation
    )
    
    # Use train directory for both train and validation (with validation_split)
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical',
        subset='training',
        shuffle=True,
        seed=SEED
    )
    
    val_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode='categorical',
        subset='validation',
        shuffle=False,
        seed=SEED
    )
    
    return train_generator, val_generator, train_generator.class_indices

print("✓ Data generator function ready")

In [ ]:
# ------------------ 4. DEFINE 18 TRAINING SCENARIOS ------------------

# Based on the table: 3 scenarios × 2 optimizers × 3 learning rates = 18 experiments
scenarios = []

scenario_configs = [
    # Scenario 1: 90:10 split
    {'split_ratio': 0.90, 'split_name': '90:10', 'lr': 0.1, 'epochs': 15},
    {'split_ratio': 0.90, 'split_name': '90:10', 'lr': 0.01, 'epochs': 30},
    {'split_ratio': 0.90, 'split_name': '90:10', 'lr': 0.001, 'epochs': 45},
    
    # Scenario 2: 80:20 split
    {'split_ratio': 0.80, 'split_name': '80:20', 'lr': 0.1, 'epochs': 15},
    {'split_ratio': 0.80, 'split_name': '80:20', 'lr': 0.01, 'epochs': 30},
    {'split_ratio': 0.80, 'split_name': '80:20', 'lr': 0.001, 'epochs': 45},
    
    # Scenario 3: 70:30 split
    {'split_ratio': 0.70, 'split_name': '70:30', 'lr': 0.1, 'epochs': 15},
    {'split_ratio': 0.70, 'split_name': '70:30', 'lr': 0.01, 'epochs': 30},
    {'split_ratio': 0.70, 'split_name': '70:30', 'lr': 0.001, 'epochs': 45},
]

optimizers_list = ['Adam', 'SGD']

# Generate all 18 combinations
scenario_id = 1
for sc in scenario_configs:
    for opt in optimizers_list:
        scenarios.append({
            'id': scenario_id,
            'split_ratio': sc['split_ratio'],
            'split_name': sc['split_name'],
            'optimizer': opt,
            'learning_rate': sc['lr'],
            'epochs': sc['epochs']
        })
        scenario_id += 1

print(f"✓ Created {len(scenarios)} training scenarios")
print("\nScenario Summary:")
for i, s in enumerate(scenarios[:3], 1):
    print(f"  Scenario {s['id']}: {s['split_name']} split, {s['optimizer']}, LR={s['learning_rate']}, Epochs={s['epochs']}")
print(f"  ... and {len(scenarios)-3} more scenarios")

In [ ]:
# ------------------ 5. BUILD FEATURE EXTRACTION MODEL ------------------

def build_feature_extraction_model(num_classes, img_size, optimizer_name, learning_rate, dropout_rate=0.5):
    """
    Build MobileNetV3Large model for feature extraction (frozen base).
    
    Args:
        num_classes: Number of output classes
        img_size: Input image size
        optimizer_name: 'Adam' or 'SGD'
        learning_rate: Learning rate for optimizer
        dropout_rate: Dropout rate before output layer
    
    Returns:
        Compiled Keras model
    """
    # Load pre-trained MobileNetV3Large without top layers
    base_model = MobileNetV3Large(
        weights='imagenet',
        include_top=False,
        input_shape=(img_size, img_size, 3)
    )
    
    # Freeze all layers in base model (feature extraction)
    for layer in base_model.layers:
        layer.trainable = False
    
    # Build custom classification head
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dropout(dropout_rate),
        layers.Dense(num_classes, activation='softmax', name='output')
    ])
    
    # Select optimizer
    if optimizer_name == 'Adam':
        opt = optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_name == 'SGD':
        opt = optimizers.SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        raise ValueError(f"Unknown optimizer: {optimizer_name}")
    
    # Compile model
    model.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

print("✓ Model builder function ready")

In [ ]:
# ------------------ 6. TRAINING LOOP FOR ALL SCENARIOS ------------------

results = []
best_val_acc = float("-inf")
best_model_path = None
best_scenario_id = None

print("=" * 70)
print(f"STARTING TRAINING FOR {len(scenarios)} SCENARIOS")
print("=" * 70)

for scenario in scenarios:
    print(f"\n{'='*70}")
    print(f"SCENARIO {scenario['id']}/{len(scenarios)}")
    print(f"  Split: {scenario['split_name']}")
    print(f"  Optimizer: {scenario['optimizer']}")
    print(f"  Learning Rate: {scenario['learning_rate']}")
    print(f"  Epochs: {scenario['epochs']}")
    print(f"{'='*70}")
    
    try:
        # Create data generators for this split ratio
        train_gen, val_gen, class_indices = create_data_generators(
            TRAIN_DIR,
            TEST_DIR,
            train_ratio=scenario['split_ratio'],
            img_size=config.IMG_SIZE_MOBILE,
            batch_size=config.BATCH_SIZE
        )
        
        print(f"  Training samples: {train_gen.samples}")
        print(f"  Validation samples: {val_gen.samples}")
        
        # Build model
        model = build_feature_extraction_model(
            num_classes=config.NUM_CLASSES,
            img_size=config.IMG_SIZE_MOBILE,
            optimizer_name=scenario['optimizer'],
            learning_rate=scenario['learning_rate'],
            dropout_rate=config.DROPOUT_RATE
        )
        
        # Callbacks
        early_stop = callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=1
        )
        
        # Train model
        history = model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=scenario['epochs'],
            callbacks=[early_stop],
            verbose=1
        )
        
        # Extract final metrics
        final_train_acc = history.history['accuracy'][-1]
        final_train_loss = history.history['loss'][-1]
        final_val_acc = history.history['val_accuracy'][-1]
        final_val_loss = history.history['val_loss'][-1]
        
        # Store results
        result = {
            'scenario_id': scenario['id'],
            'split_ratio': scenario['split_name'],
            'optimizer': scenario['optimizer'],
            'learning_rate': scenario['learning_rate'],
            'epochs': scenario['epochs'],
            'train_accuracy': final_train_acc,
            'train_loss': final_train_loss,
            'val_accuracy': final_val_acc,
            'val_loss': final_val_loss,
            'history': history.history
        }
        results.append(result)
        
        print(f"\n  ✓ Scenario {scenario['id']} completed!")
        print(f"    Final Train Accuracy: {final_train_acc:.4f}")
        print(f"    Final Val Accuracy: {final_val_acc:.4f}")
        
        # Save best model as training progresses
        if final_val_acc > best_val_acc:
            best_val_acc = final_val_acc
            best_scenario_id = scenario['id']
            best_model_path = os.path.join(config.OUTPUT_DIR, 'best_model.h5')
            model.save(best_model_path)
            print(f"    ✓ New best model saved: {best_model_path}")
        
        # Clear memory
        del model
        del train_gen
        del val_gen
        tf.keras.backend.clear_session()
        
    except Exception as e:
        print(f"\n  ✗ Scenario {scenario['id']} failed: {str(e)}")
        results.append({
            'scenario_id': scenario['id'],
            'split_ratio': scenario['split_name'],
            'optimizer': scenario['optimizer'],
            'learning_rate': scenario['learning_rate'],
            'epochs': scenario['epochs'],
            'error': str(e)
        })

print(f"\n{'='*70}")
print(f"TRAINING COMPLETED: {len([r for r in results if 'val_accuracy' in r])}/{len(scenarios)} scenarios successful")
print(f"{'='*70}")

In [ ]:
# ------------------ 7. PREPARE RESULTS DATAFRAME ------------------

# Filter successful results
successful_results = [r for r in results if 'val_accuracy' in r]

if len(successful_results) == 0:
    print("⚠ No successful training runs to analyze!")
else:
    # Create DataFrame
    df_results = pd.DataFrame([{
        'Scenario_ID': r['scenario_id'],
        'Split_Ratio': r['split_ratio'],
        'Optimizer': r['optimizer'],
        'Learning_Rate': r['learning_rate'],
        'Epochs': r['epochs'],
        'Train_Accuracy': r['train_accuracy'],
        'Val_Accuracy': r['val_accuracy'],
        'Train_Loss': r['train_loss'],
        'Val_Loss': r['val_loss']
    } for r in successful_results])
    
    # Sort by validation accuracy
    df_results = df_results.sort_values('Val_Accuracy', ascending=False).reset_index(drop=True)
    
    # Save to CSV
    results_csv = os.path.join(config.OUTPUT_DIR, 'scenario_results.csv')
    df_results.to_csv(results_csv, index=False)
    
    # Save detailed history to JSON
    history_json = os.path.join(config.OUTPUT_DIR, 'training_history.json')
    with open(history_json, 'w') as f:
        json.dump(successful_results, f, indent=2, default=str)
    
    print(f"\n✓ Results saved:")
    print(f"  - CSV: {results_csv}")
    print(f"  - JSON: {history_json}")
    
    # Display top 10
    print(f"\n{'='*70}")
    print("TOP 10 SCENARIOS BY VALIDATION ACCURACY")
    print(f"{'='*70}")
    print(df_results.head(10).to_string(index=False))
    print(f"{'='*70}")

In [ ]:
# ------------------ 8. VISUALIZATION: TOP 10 BAR CHART ------------------

if len(successful_results) > 0:
    # Get top 10 scenarios
    top_10 = df_results.head(10).copy()
    
    # Create figure
    plt.figure(figsize=(14, 8))
    
    # Create labels with all info
    top_10['Label'] = top_10.apply(
        lambda x: f"S{x['Scenario_ID']}: {x['Split_Ratio']}, {x['Optimizer']}, LR={x['Learning_Rate']}", 
        axis=1
    )
    
    # Create horizontal bar chart
    bars = plt.barh(range(len(top_10)), top_10['Val_Accuracy'], color='steelblue', alpha=0.8)
    
    # Add value labels on bars
    for i, (bar, val) in enumerate(zip(bars, top_10['Val_Accuracy'])):
        plt.text(val + 0.005, i, f'{val:.4f}', va='center', fontsize=10, fontweight='bold')
    
    plt.yticks(range(len(top_10)), top_10['Label'])
    plt.xlabel('Validation Accuracy', fontsize=12, fontweight='bold')
    plt.title('Top 10 Scenarios by Validation Accuracy', fontsize=14, fontweight='bold', pad=20)
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    
    # Save figure
    fig_path = os.path.join(config.OUTPUT_DIR, 'top_10_scenarios.png')
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    print(f"\n✓ Top 10 bar chart saved: {fig_path}")
    plt.show()
else:
    print("⚠ No results to visualize")

In [ ]:
# ------------------ 9. VISUALIZATION: TOP 3 TRAINING CURVES ------------------

if len(successful_results) >= 3:
    top_3_ids = df_results.head(3)['Scenario_ID'].tolist()
    top_3_results = [r for r in successful_results if r['scenario_id'] in top_3_ids]
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    for idx, result in enumerate(top_3_results):
        ax = axes[idx]
        history = result['history']
        epochs_range = range(1, len(history['accuracy']) + 1)
        
        # Plot accuracy
        ax.plot(epochs_range, history['accuracy'], 'b-', label='Train Accuracy', linewidth=2)
        ax.plot(epochs_range, history['val_accuracy'], 'r-', label='Val Accuracy', linewidth=2)
        
        ax.set_title(
            f"Scenario {result['scenario_id']}: {result['split_ratio']}\n"
            f"{result['optimizer']}, LR={result['learning_rate']}, Val Acc={result['val_accuracy']:.4f}",
            fontsize=11, fontweight='bold'
        )
        ax.set_xlabel('Epoch', fontsize=10)
        ax.set_ylabel('Accuracy', fontsize=10)
        ax.legend(loc='lower right')
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save figure
    fig_path = os.path.join(config.OUTPUT_DIR, 'top_3_training_curves.png')
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    print(f"✓ Top 3 training curves saved: {fig_path}")
    plt.show()
else:
    print("⚠ Not enough results to plot top 3")

In [ ]:
# ------------------ 10. VISUALIZATION: CONFUSION MATRIX FOR BEST SCENARIO ------------------

if len(successful_results) > 0:
    # Get best scenario
    best_scenario_id = df_results.iloc[0]['Scenario_ID']
    best_result = [r for r in successful_results if r['scenario_id'] == best_scenario_id][0]
    
    print(f"\n{'='*70}")
    print(f"EVALUATING BEST SCENARIO: {best_scenario_id}")
    print(f"  Split: {best_result['split_ratio']}")
    print(f"  Optimizer: {best_result['optimizer']}")
    print(f"  Learning Rate: {best_result['learning_rate']}")
    print(f"  Val Accuracy: {best_result['val_accuracy']:.4f}")
    print(f"{'='*70}")
    
    if isinstance(best_result['split_ratio'], str):
        split_num = float(best_result['split_ratio'].split(':')[0]) / 100.0
    else:
        split_num = best_result['split_ratio']
    
    # Recreate data generators for best scenario
    train_gen, val_gen, class_indices = create_data_generators(
        TRAIN_DIR,
        TEST_DIR,
        train_ratio=split_num,
        img_size=config.IMG_SIZE_MOBILE,
        batch_size=config.BATCH_SIZE
    )
    
    # Load best model if available; otherwise retrain and save
    if best_model_path and os.path.exists(best_model_path):
        best_model = tf.keras.models.load_model(best_model_path)
        print(f"\n✓ Loaded best model from: {best_model_path}")
    else:
        best_model = build_feature_extraction_model(
            num_classes=config.NUM_CLASSES,
            img_size=config.IMG_SIZE_MOBILE,
            optimizer_name=best_result['optimizer'],
            learning_rate=best_result['learning_rate'],
            dropout_rate=config.DROPOUT_RATE
        )
        
        early_stop = callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True,
            verbose=0
        )
        
        best_model.fit(
            train_gen,
            validation_data=val_gen,
            epochs=best_result['epochs'],
            callbacks=[early_stop],
            verbose=0
        )
        
        best_model_path = os.path.join(config.OUTPUT_DIR, 'best_model.h5')
        best_model.save(best_model_path)
        print(f"\n✓ Best model saved: {best_model_path}")
    
    # Generate predictions
    val_gen.reset()
    y_pred = best_model.predict(val_gen, verbose=0)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = val_gen.classes[:len(y_pred_classes)]
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred_classes)
    
    # Get class names
    class_names = list(class_indices.keys())
    
    # Plot confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'label': 'Count'})
    plt.title(f'Confusion Matrix - Best Scenario {best_scenario_id}\n'
              f'{best_result["split_ratio"]}, {best_result["optimizer"]}, LR={best_result["learning_rate"]}',
              fontsize=13, fontweight='bold', pad=15)
    plt.ylabel('True Label', fontsize=11, fontweight='bold')
    plt.xlabel('Predicted Label', fontsize=11, fontweight='bold')
    plt.tight_layout()
    
    # Save figure
    fig_path = os.path.join(config.OUTPUT_DIR, 'best_scenario_confusion_matrix.png')
    plt.savefig(fig_path, dpi=300, bbox_inches='tight')
    print(f"\n✓ Confusion matrix saved: {fig_path}")
    plt.show()
    
    # Print classification report
    print("\nClassification Report:")
    print(classification_report(y_true, y_pred_classes, target_names=class_names))
    
    # Cleanup
    del best_model
    tf.keras.backend.clear_session()
    
else:
    print("⚠ No results to evaluate")

In [ ]:
# ------------------ 11. FINAL SUMMARY ------------------

if len(successful_results) > 0:
    print(f"\n{'='*70}")
    print("EXPERIMENT SUMMARY")
    print(f"{'='*70}")
    print(f"Total Scenarios: {len(scenarios)}")
    print(f"Successful Runs: {len(successful_results)}")
    print(f"Failed Runs: {len(scenarios) - len(successful_results)}")
    print(f"\nBest Scenario: {df_results.iloc[0]['Scenario_ID']}")
    print(f"  Configuration: {df_results.iloc[0]['Split_Ratio']}, "
          f"{df_results.iloc[0]['Optimizer']}, LR={df_results.iloc[0]['Learning_Rate']}")
    print(f"  Validation Accuracy: {df_results.iloc[0]['Val_Accuracy']:.4f}")
    print(f"\nWorst Scenario: {df_results.iloc[-1]['Scenario_ID']}")
    print(f"  Configuration: {df_results.iloc[-1]['Split_Ratio']}, "
          f"{df_results.iloc[-1]['Optimizer']}, LR={df_results.iloc[-1]['Learning_Rate']}")
    print(f"  Validation Accuracy: {df_results.iloc[-1]['Val_Accuracy']:.4f}")
    
    # Performance comparison by optimizer
    print(f"\n{'='*70}")
    print("PERFORMANCE BY OPTIMIZER")
    print(f"{'='*70}")
    for opt in ['Adam', 'SGD']:
        opt_results = df_results[df_results['Optimizer'] == opt]
        if len(opt_results) > 0:
            print(f"{opt}:")
            print(f"  Mean Val Accuracy: {opt_results['Val_Accuracy'].mean():.4f}")
            print(f"  Best Val Accuracy: {opt_results['Val_Accuracy'].max():.4f}")
            print(f"  Worst Val Accuracy: {opt_results['Val_Accuracy'].min():.4f}")
    
    # Performance comparison by split ratio
    print(f"\n{'='*70}")
    print("PERFORMANCE BY SPLIT RATIO")
    print(f"{'='*70}")
    for split in ['90:10', '80:20', '70:30']:
        split_results = df_results[df_results['Split_Ratio'] == split]
        if len(split_results) > 0:
            print(f"{split}:")
            print(f"  Mean Val Accuracy: {split_results['Val_Accuracy'].mean():.4f}")
            print(f"  Best Val Accuracy: {split_results['Val_Accuracy'].max():.4f}")
    
    print(f"\n{'='*70}")
    print("All outputs saved to:", config.OUTPUT_DIR)
    print(f"{'='*70}")
    print("\n✓ Multi-scenario transfer learning experiment completed successfully!")
else:
    print("\n⚠ No successful results to summarize")